# Fontes de Erros

Neste *notebook* vamos aprender sobre as principais fontes de erros em métodos numéricos. Ao utilizar métodos numéricos e computadores de precisão finita para resolver problemas majoritariamente da engenharia ou ciência, nos deparamos, inevitavelmente, com duas principais fontes de erros: erro de truncamento e erro de arredondamento. A combinação de tais erros é chamada de erro numérico.

É importante conhecer suas naturezas e ter uma noção de suas respectivas magnitudes, em qualquer problema que envolva métodos numéricos ou computação de precisão finita.

## Erro de Truncamento
---

É o erro causado pela diferença entre a solução matemática exata e a solução calculada a partir de uma equação aproximada, geralmente utilizada para simplificar o cálculo a ser realizado. Para calcular tal erro, considera-se que os números têm precisão infinita.

Um exemplo clássico a cerca do erro de truncamento é a aproximação da função exponencial por sua Série de Taylor truncada. Vamos começar expandindo a função exponencial em sua Série de Taylor:

$$ e^x = \sum_{i = 0}^{\infty} \frac{x^i}{i!} = 1 + x + \frac{x^2}{2} + ... + \frac{x^k}{k!} + ... $$

Podemos realizar a seguinte aproximação:

$$ e^x \approx 1 + x + \frac{x^2}{2} $$

Ao calcular a função exponencial utilizando a fórmula acima, existirá um erro, mesmo que as operações sejam realizadas com precisão infinita. Tal erro é chamado de erro de truncamento.

Chamaremos o erro de truncamento de $E_T$. Para este caso em particular, como conhecemos a função exata e a função aproximada, temos:

$$ E_T = \sum_{i = 0}^{\infty}\frac{x^i}{i!} - (1 + x + \frac{x^2}{2}) = \sum_{i = 3}^{\infty}\frac{x^i}{i!} $$

## Erro de Arredondamento
---

É o erro causado devido ao fato de os computadores possuirem uma precisão finita para representar números reais e realizar cálculos entre eles.

Qualquer número irracional, como $\pi$ ou $\sqrt{2}$, necessitaria de um espaço infinito de memória para ser representado com exatidão por um computador, no entanto, os recursos computacionais são finitos e tal representação é inviável, sendo necessário representar tais números de forma aproximada.

O erro causado por tais aproximações é chamado de erro de arredondamento. Para entender melhor o comportamento deste erro, precisamos entender como um computador representa um número de ponto-flutuante.

### Pontos-flutuantes de dupla precisão

O [IEEE 754](https://en.wikipedia.org/wiki/IEEE_754) define padrões para a representação de pontos-flutuantes de precisão simples (32 bits) e dupla (64 bits), entre outras. Todos os padrões seguem o mesmo fundamento, como o Python 3 adota o ponto-flutuante de dupla precisão como formato padrão para representação de números reias, vamos utilizá-lo como exemplo.

Pontos-flutuantes de dupla precisão, 64 bits, temos a seguinte configuração:

<img src="Material Complementar/double.jpg" width="500" />

$$ \# = (-1)^s \times 2^{e - 1023} \times 1.f $$

Onde:

$$ \begin{cases}
s \in [0, 1], & \text{bit de sinal}, \\
e \in [0, 2047], & \text{expoente deslocado}, \\
f = \text{número base a ser multiplicado pela exponencial}, & \text{mantissa de 52 bits}.
\end{cases} $$

#### Números especiais

* Menor expoente: $e = 0$  
    Representa números denormais, onde a mantissa é modificada da forma: $ 1.f \rightarrow 0.f $


* Maior expoente: $e = 111 1111 1111_2$  
    $ \begin{cases}
        \pm\infty, & \text{se } m = 0, \\
        NaN, & \text{se } m \neq 0.
    \end{cases} $


* Intervalo válido de expoentes: $ -1022 \leq e-1023 \leq 1023 $


* Menor número positivo não-denormal:  
    $ \begin{cases}
        s = 0, \\
        e = 1, \\
        f = 0.
    \end{cases} $      
    $ \# = (-1)^0 \times 2^{-1022} \times 1.0 = 2^{-1022} \approx 2.2 \times 10^{-308} $


* Maior número positivo:  
    $ \begin{cases}
        s = 0, \\
        e = 11111111110_2 = 2046, \\
        f = 111...111_2.
    \end{cases} $      
    $ \# = (-1)^0 \times 2^{1023} \times (1.111...111_2) = 2^{1023} \times (2 - 2^{-52}) \approx 1.8 \times 10^{308} $


* Zero:  
    $ \begin{cases}
        s = 0, \\
        e = 0, \\
        f = 0.
    \end{cases} $

### Épsilon da Máquina

Ao realizarmos operações entre números de ponto-flutuante com magnitudes muito distintas, existe a possibilidade de o menor número ser descartado por não poder ser representado na mantissa do maior número. Desta forma, queremos identificar qual o menos número positivo que somado ao número 1 ($ s = 0, e = 1023, m = 0$), retorne um número diferente de 1. Em um ponto-flutuante de dupla precisão, a mantissa contém 52 bits, desse modo, o menor número positivo maior que 1 é o 1 seguido de 51 zeros e por fim o número 1:

$$ 1.0000000000000000000000000000000000000000000000000001_2 =  1 + 2^{-52} $$

Ou seja, um número de dupla precisão cresce, no mínimo, em passos proporcionais a $2^{-52}$ vezes o seu tamanho original. Tal valor é chamado de _épsilon da máquina_ e depende unicamente do padrão utilizado para representar números de ponto-flutuante.

Um exemplo clássico para mostrar o épsilon da máquina é tentar somar um número menor que $2^{-52}$ ao número 1:

In [1]:
n = 1.0
print('%.20f' % (n + 2**-52))
print('%.20f' % (n + 2**-53))

1.00000000000000022204
1.00000000000000000000
